# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Chelsea Brammer https://github.com/chelsbrammer/requests-JSON-and_NLP

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

In [2]:
# Create and activate a Python virtual environment. 
# Before starting the project, try all these imports FIRST
# Address any errors you get running this code cell 
# by installing the necessary packages into your active Python environment.
# Try to resolve issues using your materials and the web.
# If that doesn't work, ask for help in the discussion forums.
# You can't complete the exercises until you import these - start early! 
# We also import json and pickle (included in the Python Standard Library).

import json
import pickle

import requests
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed.')
!pip list

All prereqs installed.
Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.2
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asgiref                       3.6.0
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkcharts       

#### Question 1

1. The following code accesses the https://pypi.org/project/lyricsgenius/ public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [27]:
import lyricsgenius

import lyricsgenius
genius = lyricsgenius.Genius("BU4EfwSV70IvPZTVUs5DaoO4tcCCncs8ewdd2EtKFi6miAQ7cRhzstjESxfTDinG")

artist = genius.search_artist("Jimmy Eat World", max_songs=3, sort="title")
print(artist.songs)

song = artist.song("The Sweetness")
print(song.lyrics)

artist.add_song(song)

artist.save_lyrics()

song_desc = {
    'artist': 'Jimmy Eat World',
    'title': 'The Sweetness',
    'lyrics': song.lyrics
}

with open ('the_sweetness.json', 'w') as f:
    json.dump(song_desc, f)

Searching for songs by Jimmy Eat World...

Song 1: "12/23/95"
Song 2: "12.23.95 (Phoenix Sessions, 2021)"
Song 3: "23"

Reached user-specified song limit (3).
Done. Found 3 songs.
[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]
Searching for "The Sweetness" by Jimmy Eat World...
Done.
2 ContributorsSweetness (Live At The 9:30 Club / 2002) LyricsIf you're listening, whoa
Sing it back, whoa
String from your tether unwinds, whoa, whoa
(String from your tether unwinds)
Up and outward to bind, whoa, whoa
(Up and outward to bind)

I was spinning free, whoa
With a little sweet and simple numbing me

Are you listening, whoa
Sing it back, whoa
So tell me, what do I need, whoa, whoa
(Tell me, what do I need)
When words lose their meaning, whoa, whoa
(When words lose their meaning)

I was spinning free, whoa
With a little sweet and simple numbing me
Yeah, stumble 'til you crawl, whoa
Sinking into sweet uncertainty

(Are you listening, are you listening?)
If you're listening,

#### Question 2

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [37]:
with open('the_sweetness.json', 'r') as f:
    data = json.load(f)

print(data)

print(song.lyrics)

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")

doc = nlp(song.lyrics)
polarity_score = doc._.polarity

print("Polarity Score is:", polarity_score)

# Given that the polarity score is greater than 0, the lyrics have a more positive connotation.

{'artist': 'Jimmy Eat World', 'title': 'The Sweetness', 'lyrics': "2 ContributorsSweetness (Live At The 9:30 Club / 2002) LyricsIf you're listening, whoa\nSing it back, whoa\nString from your tether unwinds, whoa, whoa\n(String from your tether unwinds)\nUp and outward to bind, whoa, whoa\n(Up and outward to bind)\n\nI was spinning free, whoa\nWith a little sweet and simple numbing me\n\nAre you listening, whoa\nSing it back, whoa\nSo tell me, what do I need, whoa, whoa\n(Tell me, what do I need)\nWhen words lose their meaning, whoa, whoa\n(When words lose their meaning)\n\nI was spinning free, whoa\nWith a little sweet and simple numbing me\nYeah, stumble 'til you crawl, whoa\nSinking into sweet uncertainty\n\n(Are you listening, are you listening?)\nIf you're listening, whoa\n(If you're listening, are you listening?)\nSing it back, whoa\n(If you're listening, are you listening?)\nAnd I'm still running away, whoa, whoa\n(I'm still running away, whoa, whoa)\nI won't play your hide-and-

#### Question 3

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [61]:
def get_lyrics(artist, song, f):
    genius = lyricsgenius.Genius("BU4EfwSV70IvPZTVUs5DaoO4tcCCncs8ewdd2EtKFi6miAQ7cRhzstjESxfTDinG")
    artist = genius.search_artist(artist, max_songs=1,get_full_info=False)
    song = artist.song(song)
    with open(f+' song_lyrics.json', 'w') as f:
        json.dump(song.lyrics, f)

get_lyrics("Jimmy Eat World", "The Middle", "the_middle")
get_lyrics("Led Zeppelin", "Whole Lotta Love", "whole_lotta_love")
get_lyrics("Red Hot Chili Peppers", "Scar Tissue", "scar_tissue")
get_lyrics("Sublime", "What I Got", "what_I_got")

Searching for songs by Jimmy Eat World...

Song 1: "The Middle"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for songs by Led Zeppelin...

Song 1: "Stairway to Heaven"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Whole Lotta Love" by Led Zeppelin...
Done.
Searching for songs by Red Hot Chili Peppers...

Song 1: "Californication"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Scar Tissue" by Red Hot Chili Peppers...
Done.
Searching for songs by Sublime...

Song 1: "Santeria"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "What I Got" by Sublime...
Done.


#### Question 4

4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [63]:
def sentiment_analysis(f):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    with open(f, 'r') as f:
        lyrics = json.load(f)
    doc = nlp(lyrics)
    polarity = doc._.blob.polarity
    
    print(f'Polarity scores of {f.name}: {polarity}')
#getting song polarity scores by json file pf lyrics    
sentiment_analysis('scar_tissue song_lyrics.json')
sentiment_analysis('the_middle song_lyrics.json')
sentiment_analysis('what_i_got song_lyrics.json')
sentiment_analysis('whole_lotta_love song_lyrics.json')

# The Middle by Jimmy Eat World, What I Got by Sublime and Whole Lotta Love by Led Zeppelin all got positive polarity scores which I think
#aligns with their polarity scores. Scar Tissue by Red Hot Chili Peppers got a slightly negative polarity score and I think only because
#the lyrics 'scar tissue I wish you saw' and 'with the birds I"ll share this lonely view' reflect hardships people carry but keep to themselves.

Polarity scores of scar_tissue song_lyrics.json: -0.05161290322580645
Polarity scores of the_middle song_lyrics.json: 0.1861786361786362
Polarity scores of what_i_got song_lyrics.json: 0.02480519480519481
Polarity scores of whole_lotta_love song_lyrics.json: 0.34947089947089943
